In [7]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
from timeit import default_timer as timer
from _collections import OrderedDict
import shutil
from pprint import pprint
start = timer()
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(PATH)
from utilities.file_location import FileLocationManager
from utilities.sqlcontroller import SqlController
from utilities.utilities_cvat_neuroglancer import get_structure_number, NumpyToNeuroglancer, get_segment_properties
from utilities.utilities_affine import align_point_sets, load_atlas_data, estimate_structure_centers


In [3]:
animal = 'DK52'
fileLocationManager = FileLocationManager(animal)
OUTPUT_DIR = os.path.join(fileLocationManager.neuroglancer_data, 'atlas')
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

atlas_data = load_atlas_data()
source_centers = estimate_structure_centers(atlas_data, animal)
# DK52 structure centers
output_centers = {
    '12N': [46488, 18778, 242],
    '5N_L': [38990, 20019, 172],
    '5N_R': [39184, 19027, 315],
    '7N_L': [42425, 23190, 166],
    '7N_R': [42286, 22901, 291]
}
structures = sorted(output_centers.keys())
source_point_set = np.array([source_centers[s] for s in structures]).T
# Unify to 1 mum scale in all axes
source_scale = np.diagflat([10, 10, 20])
source_point_set = source_scale @ source_point_set
output_point_set = np.array([output_centers[s] for s in structures]).T

# Unify to 1 mum scale in all axes
output_scale = np.diagflat([0.325, 0.325, 20])
output_point_set = output_scale @ output_point_set
r_auto, t_auto = align_point_sets(source_point_set, output_point_set)
t_auto_out = (np.linalg.inv(output_scale) @ t_auto).T


resolution: 0.325
width: 65000
height: 36000
box center: [1056.25  585.    242.5 ]


In [11]:
sql_controller = SqlController(animal)
thumbnail_dir = os.path.join(fileLocationManager.prep, 'CH1', 'thumbnail_aligned')

print(f'resolution: {sql_controller.scan_run.resolution}')
print(f'width: {sql_controller.scan_run.width}')
print(f'height: {sql_controller.scan_run.height}')
box_w = sql_controller.scan_run.width * sql_controller.scan_run.resolution / 10  # 10 mum scale
box_h = sql_controller.scan_run.height * sql_controller.scan_run.resolution / 10  # 10 mum scale
box_z = len(os.listdir(thumbnail_dir))  # 20 mum scale
box_center = np.array([box_w, box_h, box_z]) / 2

atlasV7_volume = np.zeros((int(box_h), int(box_w), int(box_z)), dtype=np.uint8)
print('Shape of atlas volume', atlasV7_volume.shape)


resolution: 0.325
width: 65000
height: 36000
Shape of atlas volume (1170, 2112, 485)


In [12]:
for structure, (volume, origin) in sorted(atlas_data.items()):
    print(str(structure).ljust(6),end=": ")
    print(volume.shape)
    x,y,z = origin
    arr = np.array([y,x,z])
    input = arr + t_auto_out
    print(input)
    continue
    results = np.dot(r_auto, input.T).reshape(3,1)
    y = results[0][0]
    x = results[1][0]
    z = results[2][0]
    x_start = int( round(x + col_length / 2))
    y_start = int( round(y + row_length / 2))
    z_start = int( round(z / 2 + z_length / 2))
    x_end = x_start + volume.shape[0]
    y_end = y_start + volume.shape[1]
    z_end = int( z_start + (volume.shape[2] + 1) / 2)
    print('Row', str(y_start).rjust(4), str(y_end).rjust(4),'Col', str(x_start).rjust(4), str(x_end).rjust(4),
          'Z', str(z_start).rjust(4), str(z_end).rjust(4), end=" ")

        
    z_indices = [z for z in range(volume.shape[2]) if z % 2 == 0]
    volume = volume[:, :, z_indices]
    volume = np.swapaxes(volume, 0, 1)

    try:
        atlasV7_volume[y_start:y_end, x_start:x_end, z_start:z_end] += volume
    except ValueError as ve:
        print("Error", end=" ")

    print()
# rotation puts in view with wide x and shallow  y
atlasV7_volume = np.rot90(atlasV7_volume, 1)
# fliplr and flip axis=1 did not work, axis=0 works
atlasV7_volume = np.flip(atlasV7_volume, axis=0)
#atlasV7_volume = affine_transform(atlasV7_volume, R)
print('Done with big volume:', atlasV7_volume.shape)

10N_L : (3,)


ValueError: too many values to unpack (expected 3)

In [ ]:
atlasV7_volume.shape

In [ ]:
%%time
ng = NumpyToNeuroglancer(atlasV7_volume, [10000, 10000, 20000])
ng.preview()
ng.init_precomputed(OUTPUT_DIR)
ng.add_segment_properties(get_segment_properties())
ng.add_downsampled_volumes()
ng.add_segmentation_mesh()